In [1]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.metrics import r2_score as R2
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score as r2
from copy import deepcopy
import utils
from unet import UNet_nobatchnorm
from scipy.stats import pearsonr
#JU's addtion to automate inputs and outputs
import helper_functions as hf
torch.cuda.set_device(2)
import os
def save_fn(var_input_list, var_output_list):
    var_input_join  = '_and_'.join(var_input_list)
    var_output_join = '_and_'.join(var_output_list)
    return '{}_to_{}'.format(var_input_join, var_output_join)

In [2]:
print(torch.__version__)
print(torch.version.cuda)

1.13.0
11.7


In [3]:
maxEpochs =  300#small number is taken for debugging
nensemble = 10 #How many training sessions are run for each configuration ***
Nbase = 16

In [4]:
!nvidia-smi #GPU usage should be maxed out during training; tune batch_size according to that

Fri Mar 28 11:27:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.02              Driver Version: 555.42.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:03:00.0 Off |                    0 |
| N/A   47C    P0             67W /  500W |       5MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [5]:
root_dir = '/work/uo0780/u241359/project_tide_synergy/data/'
nctrains, nctest = hf.load_data_from_nc_as_lists(root_dir)

device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print ('Running on ', device)

Ntrain = np.sum([nc.dimensions['time_counter'].size for nc in nctrains], axis = 0)
Ntest = np.sum([nc.dimensions['time_counter'].size for nc in nctest], axis = 0)

print('number of training records:', Ntrain)
print('number of testing records:', Ntest)

numTrainFiles = len(nctrains)
numRecsFile = nctrains[0].dimensions['time_counter'].size #How many snapshots in time in each data set there is
print (numRecsFile)

Running on  cuda:2
number of training records: 600
number of testing records: 150
150


In [6]:
def hwvorticity(u, v, dgrid = 4000):
    return (np.gradient(v, axis =2) - np.gradient(u, axis =1))/dgrid

def hwdivergence(u, v, dgrid = 4000):
    return (np.gradient(u, axis =2) + np.gradient(v, axis =1))/dgrid

def preload_data_vortdiv(nctrains, total_records):
    #total_records = Ntrain#sum(nc.dimensions['time_counter'].size for nc in nctrains)
    #dimensions of data of the nc file.
    max_height = 722
    max_width = 258
    all_input_data = np.zeros((total_records, N_inp, max_height, max_width))*np.nan
    all_output_data = np.zeros((total_records, N_out, max_height, max_width))*np.nan
    current_index = 0
    for ncindex, ncdata in enumerate(nctrains):
        num_recs = ncdata.dimensions['time_counter'].size #how many time stamps are there in each .nc file (i.e., at each turbulence level)
        rec_slice = slice(current_index, current_index + num_recs)
        for ind, var_name in enumerate(var_input_names):
            if var_name == 'vort':
                u = np.squeeze(ncdata.variables['u_xy_ins'])
                v = np.squeeze(ncdata.variables['v_xy_ins'])
                #u.shape: (150, 722, 257); v.shape: (150, 721, 258)
                #as u and v have different number of grid points in x and y, we truncate them so that their shapes agree, enabling the simple way to compute vorticities based on finite diff.
                data_slice = hwvorticity(u[:,:-1,:], v[:,:,:-1])
            elif var_name == 'div':
                u = np.squeeze(ncdata.variables['u_xy_ins'])
                v = np.squeeze(ncdata.variables['v_xy_ins'])
                data_slice = hwdivergence(u[:,:-1,:], v[:,:,:-1])
            else:           
                data_slice = np.squeeze(ncdata.variables[var_name])
            # print('data_slice shape:')
            # print(data_slice.shape)        
            #all_input_data[rec_slice, ind, :, :] = data_slice
            
            #For some variables, the dimensions in (x, y) may be smaller than (max_height, max_width). Changing the code so that it adapts them.
            # Get the actual dimensions of data_slice
            slice_height, slice_width = data_slice.shape[-2], data_slice.shape[-1]
            # Place data_slice into the corresponding slice of all_input_data
            all_input_data[rec_slice, ind, :slice_height, :slice_width] = data_slice
    

        for ind, var_name in enumerate(var_output_names):
            data_slice = np.squeeze(ncdata.variables[var_name])
            #all_output_data[rec_slice, ind, :, :] = data_slice
            # Get the actual dimensions of data_slice
            slice_height, slice_width = data_slice.shape[-2], data_slice.shape[-1]
            # Place data_slice into the corresponding slice of all_input_data
            all_output_data[rec_slice, ind, :slice_height, :slice_width] = data_slice

        current_index += num_recs
        
    return all_input_data, all_output_data
    


# Modify the loadtrain function to pull data from preloaded memory
def loaddata_preloaded_train(index, batch_size, all_input_data, all_output_data):
    rec_slice = slice(index, index + batch_size)
    lim = 720
    width = 256
    yslice = slice(0, lim)
    xslice = slice(0, width)
    # print('mean of squared values of loaded input data:')
    # print("{0:0.32f}".format(np.nanmean(all_input_data[rec_slice, :, yslice, xslice]**2)))
    return (all_input_data[rec_slice, :, yslice, xslice], 
            all_output_data[rec_slice, :, yslice, xslice])
#Load test data as one single batch
def loaddata_preloaded_test(all_input_data, all_output_data):
    #rec_slice = slice(index, index + batch_size)
    lim = 720
    width = 256
    yslice = slice(0, lim)
    xslice = slice(0, width)
    # print('rec_slice is:')
    # print(rec_slice)
    # print('mean of squared values of loaded input data:')
    # print("{0:0.32f}".format(np.nanmean(all_input_data[rec_slice, :, yslice, xslice]**2)))
    return (all_input_data[:, :, yslice, xslice], 
            all_output_data[:, :, yslice, xslice])



In [7]:
def run_model(var_input_names, var_output_names, save_fn_prefix, N_inp, N_out, iensemble, R2_all, corr_all):
    ytest_slice = slice(0, 720)
    xtest_slice = slice(0, 256)
    rectest_slice = slice(0, 150)

    def totorch(x):
        return torch.tensor(x, dtype = torch.float).cuda()

    model = UNet_nobatchnorm(N_inp, N_out, bilinear = True, Nbase = Nbase).cuda()
    #model = torch.compile(UNet(N_inp, N_out, bilinear = True, Nbase = Nbase).cuda())

    if iensemble == 0:
        input = torch.randn(1,N_inp,256,720).to(device) 
        output = model(input)
        print('Model has ', utils.nparams(model)/1e6, ' million params')

    # for index in range(0, Ntrain, batch_size):
    #     inp, out = loadtrain_preloaded(index, batch_size, all_train_input, all_train_output)
    #     print(inp.shape, out.shape)
#         print(np.nanmean(inp**2), np.max(inp**2), inp.shape)
#         print(np.nanmean(out**2), np.max(out**2), inp.shape)

    inp_test, out_test = loaddata_preloaded_test(all_test_input, all_test_output)
    #inp, out_test = loadtest()
    # print('shapes of input and output TEST data:')
    # print(inp_test.shape, out_test.shape)
    with torch.no_grad():
        inp_test = totorch(inp_test)

    Tcycle = 10
    criterion_train  = nn.L1Loss()
    optim = torch.optim.AdamW(model.parameters(), lr=lr0, betas=(0.9, 0.999), eps=1e-8, weight_decay=1e-5*100) #increase weight_decay ***

    r2_test = np.zeros(maxEpochs)
    epochmin = []
    maxr2l = []

    learn = np.zeros(maxEpochs)
    minloss = 1000
    maxR2 = -1000
    minlosscount = 0
    perm = False

    model_best = deepcopy(model)  # Initialize before the loop for safety

    #print('Starting training loop')
    for epoch in tqdm(range(maxEpochs)):
        lr = utils.cosineSGDR(optim, epoch, T0=Tcycle, eta_min=0, eta_max=lr0, scheme = 'constant')  #captioning this seems to make the printed corr larger??***
        model.train()
        index_perm = np.arange(0, Ntrain, batch_size)
        
        if perm:
            index_perm = np.random.permutation(index_perm)
        
        for index in index_perm:
            inp, out = loaddata_preloaded_train(index, batch_size, all_train_input, all_train_output)            
#           inp, out = loadtrain(index, batch_size, nctrains)
            inp, out = totorch(inp), totorch(out)
            #continue #do this to pause the later operations to check how long it takes for the steps up to this 
            out_mod = model(inp)
            loss = criterion_train(out.squeeze(), out_mod.squeeze())
            #Set gradient to zero
            optim.zero_grad()
            #Compute gradients       
            loss.backward()
            #Update parameters with new gradient
            optim.step()
            #Record train loss
            #scheduler.step()
          
        model.eval()
        with torch.no_grad():
            #model_cpu = model.to('cpu')
            #out_mod = model_cpu(inp_test.to('cpu'))
            out_mod=model(inp_test)
            
            r2 = R2(out_test.flatten(), (out_mod).cpu().numpy().flatten())
            r2_test[epoch] = r2
            #print('Debugging: R2 of current epoch:', r2)#Debugging
            #record current best model and best predictions
            if maxR2 <  r2:
                maxR2 = r2
                epochmin.append(epoch)
                maxr2l.append(maxR2)                
                model_best = deepcopy(model)
                corr, pval = pearsonr(out_test.flatten(), (out_mod).cpu().numpy().flatten())
                print('R2:', r2, ' corr: ', corr, ' pval: ', pval)
            #model = model_cpu.to(device)

    #_, out_test = loadtest()
    model_best.eval()
    with torch.no_grad():
    #     inp_test = totorch(inp)
        model_best.to('cpu') #added by HW 
        out_mod = model_best(inp_test.to('cpu')).detach().cpu().numpy()

    R2_all[iensemble]=R2(out_test.flatten(), out_mod.flatten())
    corr_all[iensemble]=pearsonr(out_test.flatten(), out_mod.flatten())[0]
    # print('Best model R2:', R2_all[iensemble])#pearsonr(out_test.flatten(), out_mod.flatten())[0])
    # print('Best model corr:', corr_all[iensemble])#R2(out_test.flatten(), out_mod.flatten()))

    # Nx, Ny = out_test.shape[2:]; Nx, Ny

    # print(out_mod.shape, 'outout model shape')
    dr = '/work/uo0780/u241359/project_tide_synergy/trainedmodels' #'./models/to_vel'
    os.makedirs(dr, exist_ok=True) # exist_ok=True allows the function to do nothing (i.e., not raise an error) if the directory already exists.
    fstr = f'{save_fn_prefix}_rp_{iensemble}'
    PATH = dr + f'/{fstr}.pth'
    torch.save(model_best.state_dict(), PATH)

In [8]:
vi1 = 'vort'

vo1 = 'ssh_cos'
vo2 = 'ssh_sin'

save_fn_prefix  = 'any_{}_{}{}_nobatchnorm'.format(vi1, vo1, vo2)
var_input_names = [vi1]
var_output_names = [vo1, vo2]

batch_size = 100 #maximizing it so that the GPU memory maxes out. Needs to be divisible by Ntrain. Otherwise there will be size mismatch issues.

N_inp = len(var_input_names)
N_out = len(var_output_names)

lr0 = 0.005*10/batch_size #Roughly should scale inversely to batch_size

#Recording performance metrics on test data after eaching training cycle
R2_all = np.zeros(nensemble)
corr_all = np.zeros(nensemble)

nctrains, nctest = hf.load_data_from_nc_as_lists(root_dir)

all_train_input, all_train_output = preload_data_vortdiv(nctrains, Ntrain)
all_test_input, all_test_output = preload_data_vortdiv(nctest, Ntest)

#Normalize data
#Compute mean and variance for normalization
mean_input=np.nanmean(np.concatenate((all_train_input, all_test_input), axis=0),axis=(0, 2, 3))
mean_output=np.nanmean(np.concatenate((all_train_output, all_test_output), axis=0),axis=(0, 2, 3))
#Subtract the data with their means
all_train_input=all_train_input-mean_input[None, :, None, None]
all_train_output=all_train_output-mean_output[None, :, None, None]
all_test_input=all_test_input-mean_input[None, :, None, None]
all_test_output=all_test_output-mean_output[None, :, None, None]
#Compute the variances
var_input=np.nanmean((np.concatenate((all_train_input, all_test_input), axis=0))**2,axis=(0, 2, 3))
var_output=np.nanmean((np.concatenate((all_train_output, all_test_output), axis=0))**2,axis=(0, 2, 3))
print("mean and variance of all input data:")
print(mean_input,var_input)
print("mean and variance of all output data:")
print(mean_output,var_output)
#Scale the data so that they have variance of 1
all_train_input=all_train_input/np.sqrt(var_input[None, :, None, None])
all_train_output=all_train_output/np.sqrt(var_output[None, :, None, None])
all_test_input=all_test_input/np.sqrt(var_input[None, :, None, None])
all_test_output=all_test_output/np.sqrt(var_output[None, :, None, None])
#Have checked that after these operations, the data is scaled to be zero mean and variance 1.

for iensemble in np.arange(nensemble):
    run_model(var_input_names, var_output_names, save_fn_prefix, N_inp, N_out, iensemble, R2_all, corr_all)  
print('R2 from the best models in each run are:')
print(R2_all)
print('corr from the best models in each run are:')
print(corr_all)

mean and variance of all input data:
[-1.37220302e-09] [1.34097669e-10]
mean and variance of all output data:
[-5.16228102e-04 -9.83592627e-05] [9.36516511e-05 1.01456128e-04]
Model has  1.124418  million params


  0%|          | 1/300 [00:06<33:44,  6.77s/it]

R2: -0.02631400736582834  corr:  0.0196050554666634  pval:  0.0


  1%|          | 2/300 [00:12<31:19,  6.31s/it]

R2: -0.0015840129545836579  corr:  0.10069403740710732  pval:  0.0


...

R2: 0.669577843210164  corr:  0.8185910654641719  pval:  0.0


 27%|██▋       | 80/300 [07:57<23:13,  6.33s/it]

R2: 0.6697429078357849  corr:  0.8188290810233884  pval:  0.0


100%|██████████| 300/300 [26:45<00:00,  5.35s/it]


R2 from the best models in each run are:
[0.67081791 0.67331563 0.67277283 0.67680986 0.67591147 0.67683056
 0.6847498  0.66735077 0.68215699 0.66974873]
corr from the best models in each run are:
[0.81968802 0.82113936 0.82087197 0.82314147 0.82245843 0.82274272
 0.82772153 0.81708915 0.8265746  0.81883238]


In [8]:
vi1 = 'vort'
vi2 = 'T_xy_ins'

vo1 = 'ssh_cos'
vo2 = 'ssh_sin'

save_fn_prefix  = 'any_{}{}_{}{}_nobatchnorm'.format(vi1, vi2, vo1, vo2)
var_input_names = [vi1, vi2]
var_output_names = [vo1, vo2]

batch_size = 80 #maximizing it so that the GPU memory maxes out. Needs to be divisible by Ntrain. Otherwise there will be size mismatch issues.

N_inp = len(var_input_names)
N_out = len(var_output_names)

lr0 = 0.005*10/batch_size #Roughly should scale inversely to batch_size

#Recording performance metrics on test data after eaching training cycle
R2_all = np.zeros(nensemble)
corr_all = np.zeros(nensemble)

nctrains, nctest = hf.load_data_from_nc_as_lists(root_dir)

all_train_input, all_train_output = preload_data_vortdiv(nctrains, Ntrain)
all_test_input, all_test_output = preload_data_vortdiv(nctest, Ntest)

#Normalize data
#Compute mean and variance for normalization
mean_input=np.nanmean(np.concatenate((all_train_input, all_test_input), axis=0),axis=(0, 2, 3))
mean_output=np.nanmean(np.concatenate((all_train_output, all_test_output), axis=0),axis=(0, 2, 3))
#Subtract the data with their means
all_train_input=all_train_input-mean_input[None, :, None, None]
all_train_output=all_train_output-mean_output[None, :, None, None]
all_test_input=all_test_input-mean_input[None, :, None, None]
all_test_output=all_test_output-mean_output[None, :, None, None]
#Compute the variances
var_input=np.nanmean((np.concatenate((all_train_input, all_test_input), axis=0))**2,axis=(0, 2, 3))
var_output=np.nanmean((np.concatenate((all_train_output, all_test_output), axis=0))**2,axis=(0, 2, 3))
print("mean and variance of all input data:")
print(mean_input,var_input)
print("mean and variance of all output data:")
print(mean_output,var_output)
#Scale the data so that they have variance of 1
all_train_input=all_train_input/np.sqrt(var_input[None, :, None, None])
all_train_output=all_train_output/np.sqrt(var_output[None, :, None, None])
all_test_input=all_test_input/np.sqrt(var_input[None, :, None, None])
all_test_output=all_test_output/np.sqrt(var_output[None, :, None, None])
#Have checked that after these operations, the data is scaled to be zero mean and variance 1.

for iensemble in np.arange(nensemble):
    run_model(var_input_names, var_output_names, save_fn_prefix, N_inp, N_out, iensemble, R2_all, corr_all)  
print('R2 from the best models in each run are:')
print(R2_all)
print('corr from the best models in each run are:')
print(corr_all)

mean and variance of all input data:
[-1.37220302e-09  2.51429352e+01] [1.34097669e-10 3.41196182e-01]
mean and variance of all output data:
[-5.16228102e-04 -9.83592627e-05] [9.36516511e-05 1.01456128e-04]
Model has  1.124562  million params


  0%|          | 1/300 [00:06<32:24,  6.50s/it]

R2: -0.019339216775192014  corr:  0.0047771987199520865  pval:  2.0899949627486065e-276


  1%|          | 2/300 [00:12<31:26,  6.33s/it]

R2: 0.00742938492603562  corr:  0.09666751957546348  pval:  0.0


...

R2: 0.7177174585308415  corr:  0.8475109544653656  pval:  0.0


 37%|███▋      | 110/300 [10:10<18:20,  5.79s/it]

R2: 0.7188621804458135  corr:  0.8480649259055884  pval:  0.0


100%|██████████| 300/300 [25:24<00:00,  5.08s/it]


R2 from the best models in each run are:
[0.72519143 0.72034342 0.72279996 0.72211409 0.7168879  0.72347452
 0.71711332 0.71517292 0.72143852 0.71885995]
corr from the best models in each run are:
[0.85193662 0.84874994 0.85023146 0.85003446 0.8467613  0.85060491
 0.84703371 0.84568836 0.84940155 0.84806376]


In [ ]:
# vi1 = 'div'

# vo1 = 'ssh_cos'
# vo2 = 'ssh_sin'

# save_fn_prefix  = 'any_{}_{}{}_nobatchnorm'.format(vi1, vo1, vo2)
# var_input_names = [vi1]
# var_output_names = [vo1, vo2]

# batch_size = 100 #maximizing it so that the GPU memory maxes out. Needs to be divisible by Ntrain. Otherwise there will be size mismatch issues.

# N_inp = len(var_input_names)
# N_out = len(var_output_names)

# lr0 = 0.005*10/batch_size #Roughly should scale inversely to batch_size

# #Recording performance metrics on test data after eaching training cycle
# R2_all = np.zeros(nensemble)
# corr_all = np.zeros(nensemble)

# nctrains, nctest = hf.load_data_from_nc_as_lists(root_dir)

# all_train_input, all_train_output = preload_data_vortdiv(nctrains, Ntrain)
# all_test_input, all_test_output = preload_data_vortdiv(nctest, Ntest)

# #Normalize data
# #Compute mean and variance for normalization
# mean_input=np.nanmean(np.concatenate((all_train_input, all_test_input), axis=0),axis=(0, 2, 3))
# mean_output=np.nanmean(np.concatenate((all_train_output, all_test_output), axis=0),axis=(0, 2, 3))
# #Subtract the data with their means
# all_train_input=all_train_input-mean_input[None, :, None, None]
# all_train_output=all_train_output-mean_output[None, :, None, None]
# all_test_input=all_test_input-mean_input[None, :, None, None]
# all_test_output=all_test_output-mean_output[None, :, None, None]
# #Compute the variances
# var_input=np.nanmean((np.concatenate((all_train_input, all_test_input), axis=0))**2,axis=(0, 2, 3))
# var_output=np.nanmean((np.concatenate((all_train_output, all_test_output), axis=0))**2,axis=(0, 2, 3))
# print("mean and variance of all input data:")
# print(mean_input,var_input)
# print("mean and variance of all output data:")
# print(mean_output,var_output)
# #Scale the data so that they have variance of 1
# all_train_input=all_train_input/np.sqrt(var_input[None, :, None, None])
# all_train_output=all_train_output/np.sqrt(var_output[None, :, None, None])
# all_test_input=all_test_input/np.sqrt(var_input[None, :, None, None])
# all_test_output=all_test_output/np.sqrt(var_output[None, :, None, None])
# #Have checked that after these operations, the data is scaled to be zero mean and variance 1.

# for iensemble in np.arange(nensemble):
#     run_model(var_input_names, var_output_names, save_fn_prefix, N_inp, N_out, iensemble, R2_all, corr_all)  
# print('R2 from the best models in each run are:')
# print(R2_all)
# print('corr from the best models in each run are:')
# print(corr_all)

In [ ]:
# vi1 = 'vort'
# vi2 = 'div'

# vo1 = 'ssh_cos'
# vo2 = 'ssh_sin'

# save_fn_prefix  = 'any_{}{}_{}{}_nobatchnorm'.format(vi1, vi2, vo1, vo2)
# var_input_names = [vi1, vi2]
# var_output_names = [vo1, vo2]

# batch_size = 80 #maximizing it so that the GPU memory maxes out. Needs to be divisible by Ntrain. Otherwise there will be size mismatch issues.

# N_inp = len(var_input_names)
# N_out = len(var_output_names)

# lr0 = 0.005*10/batch_size #Roughly should scale inversely to batch_size

# #Recording performance metrics on test data after eaching training cycle
# R2_all = np.zeros(nensemble)
# corr_all = np.zeros(nensemble)

# nctrains, nctest = hf.load_data_from_nc_as_lists(root_dir)

# all_train_input, all_train_output = preload_data_vortdiv(nctrains, Ntrain)
# all_test_input, all_test_output = preload_data_vortdiv(nctest, Ntest)

# #Normalize data
# #Compute mean and variance for normalization
# mean_input=np.nanmean(np.concatenate((all_train_input, all_test_input), axis=0),axis=(0, 2, 3))
# mean_output=np.nanmean(np.concatenate((all_train_output, all_test_output), axis=0),axis=(0, 2, 3))
# #Subtract the data with their means
# all_train_input=all_train_input-mean_input[None, :, None, None]
# all_train_output=all_train_output-mean_output[None, :, None, None]
# all_test_input=all_test_input-mean_input[None, :, None, None]
# all_test_output=all_test_output-mean_output[None, :, None, None]
# #Compute the variances
# var_input=np.nanmean((np.concatenate((all_train_input, all_test_input), axis=0))**2,axis=(0, 2, 3))
# var_output=np.nanmean((np.concatenate((all_train_output, all_test_output), axis=0))**2,axis=(0, 2, 3))
# print("mean and variance of all input data:")
# print(mean_input,var_input)
# print("mean and variance of all output data:")
# print(mean_output,var_output)
# #Scale the data so that they have variance of 1
# all_train_input=all_train_input/np.sqrt(var_input[None, :, None, None])
# all_train_output=all_train_output/np.sqrt(var_output[None, :, None, None])
# all_test_input=all_test_input/np.sqrt(var_input[None, :, None, None])
# all_test_output=all_test_output/np.sqrt(var_output[None, :, None, None])
# #Have checked that after these operations, the data is scaled to be zero mean and variance 1.

# for iensemble in np.arange(nensemble):
#     run_model(var_input_names, var_output_names, save_fn_prefix, N_inp, N_out, iensemble, R2_all, corr_all)  
# print('R2 from the best models in each run are:')
# print(R2_all)
# print('corr from the best models in each run are:')
# print(corr_all)